In [ ]:
import pandas as pd
import numpy as np
import pipeline

import geopandas as gpd

import holoviews as hv, hvplot.pandas, panel as pn

pd.set_option("display.max_columns", 100)

In [ ]:
date = "2023-05-01"

In [ ]:
ais = pipeline.build_ais(date)

In [ ]:
sat_tiles = pipeline.build_tiles(date)

In [ ]:
chips = pipeline.build_chips(ais, sat_tiles)

In [ ]:
import numpy as np
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import logging
import datetime

from pystac_client import Client
from shapely.geometry import shape, Point
import geopandas as gpd

In [ ]:
from pystac_client import Client
import random

In [ ]:
sat_url = "https://earth-search.aws.element84.com/v1"
sat_api = Client.open(sat_url)

In [ ]:
row = chips.sample(1)
row

In [ ]:
crop_buffer = 0.02  # .01 = ~1 km

In [ ]:
tile_id = row.id.values[0]

In [ ]:
search = sat_api.search(
    collections=["sentinel-1-grd"],
    bbox=[
        row["pred_lon"] - crop_buffer,
        row["pred_lat"] - crop_buffer,
        row["pred_lon"] + crop_buffer,
        row["pred_lat"] + crop_buffer,
    ],
    ids=tile_id,
    query={
        "sar:instrument_mode": {"eq": "IW"},
    },
)

In [ ]:
item = list(search.get_items())[0]
tile_geom = shape(item.geometry)
tile_geom.bounds

In [ ]:
import odc.stac
from pystac_client import Client
from planetary_computer import sign


def get_sar_crop(row, crop_buffer=crop_buffer):

    api = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = api.search(
        collections=["sentinel-1-grd"],
        bbox=[
            row["pred_lon"] - crop_buffer,
            row["pred_lat"] - crop_buffer,
            row["pred_lon"] + crop_buffer,
            row["pred_lat"] + crop_buffer,
        ],
        datetime=row["pred_time"].isoformat(),  # Exact time of the tile
        query={"sar:instrument_mode": {"eq": "IW"}},
    )

    items = list(search.items())
    item = sign(items[0])

    ds = odc.stac.load(
        [item],
        bands=["vv", "vh"],
        bbox=[
            row["pred_lon"] - crop_buffer,
            row["pred_lat"] - crop_buffer,
            row["pred_lon"] + crop_buffer,
            row["pred_lat"] + crop_buffer,
        ],
        crs="EPSG:4326",
        resolution=0.0001,
        chunks={},
    ).isel(time=0)

    crop = ds.sel(
        latitude=slice(row["pred_lat"] + crop_buffer, row["pred_lat"] - crop_buffer),
        longitude=slice(row["pred_lon"] - crop_buffer, row["pred_lon"] + crop_buffer),
    )

    return crop

In [ ]:
image = get_sar_crop(row.iloc[0])

In [ ]:
image = image.vh.compute()

In [ ]:
image.values

In [ ]:
vmin, vmax = np.percentile(image.values[~np.isnan(image.values)], (2, 98))

In [ ]:
# Basic plot of the VV band
image.hvplot.image(
    x="longitude",
    y="latitude",
    cmap="gray",
    clim=(vmin, vmax),
    rasterize=True,
    aspect="equal",
    title="Sentinel-1 VV Backscatter",
)